In [67]:
# ================================
#  Web3 Trading Team DS Assignment
#  Complete EDA + UBM Analysis
#  Author: Luv Goel
# ================================

# --- Libraries ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime

# --- Settings ---
import seaborn as sns
sns.set_style("whitegrid")

In [68]:
# === 1. Create Required Submission Structure ===
root = "ds_LuvGoel"
csv_folder = os.path.join(root, "csv_files")
outputs_folder = os.path.join(root, "outputs")
os.makedirs(csv_folder, exist_ok=True)
os.makedirs(outputs_folder, exist_ok=True)

In [70]:
# Install and import necessary libraries
!pip install -q gdown

import gdown
import pandas as pd

# Google Drive file IDs
fear_greed = '1ESUC2mftmOr1IXhQIKsLrDNurcwi7NRX'
historical = '1uPGS5u5nNUEdgk0BvWqk3ze01nz-2mv_'

# Build direct download URLs
url1 = f'https://drive.google.com/uc?id={fear_greed}'
url2 = f'https://drive.google.com/uc?id={historical}'

# Local file paths
output1 = 'fear_greed.csv'
output2 = 'historical.csv'

# Download CSV files
gdown.download(url1, output1, quiet=False)
gdown.download(url2, output2, quiet=False)

# Load CSV files into DataFrames
fear_greed_index = pd.read_csv(output1)
historical_data = pd.read_csv(output2)

# Preview the data
print(fear_greed_index.head())
print(historical_data.head())

Downloading...
From: https://drive.google.com/uc?id=1ESUC2mftmOr1IXhQIKsLrDNurcwi7NRX
To: /content/fear_greed.csv
100%|██████████| 90.8k/90.8k [00:00<00:00, 51.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1uPGS5u5nNUEdgk0BvWqk3ze01nz-2mv_
To: /content/historical.csv
100%|██████████| 47.5M/47.5M [00:00<00:00, 134MB/s]


    timestamp  value classification        date
0  1517463000     30           Fear  2018-02-01
1  1517549400     15   Extreme Fear  2018-02-02
2  1517635800     40           Fear  2018-02-03
3  1517722200     24   Extreme Fear  2018-02-04
4  1517808600     11   Extreme Fear  2018-02-05
                                      Account  Coin  Execution Price  \
0  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9769   
1  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9800   
2  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9855   
3  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9874   
4  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9894   

   Size Tokens  Size USD Side     Timestamp IST  Start Position Direction  \
0       986.87   7872.16  BUY  02-12-2024 22:50        0.000000       Buy   
1        16.00    127.68  BUY  02-12-2024 22:50      986.524596       Buy   
2       144.09   1150.63  BUY  02-12-2024 22:50 

In [71]:
 # === 3. Data Cleaning ===
# Convert timestamps (historical_data) -> dayfirst to avoid ValueError
historical_data['Timestamp'] = pd.to_datetime(
    historical_data['Timestamp IST'],
    format="%d-%m-%Y %H:%M",
    errors='coerce'
)

In [72]:
# Convert timestamps (fear_greed_index)
fear_greed_index['timestamp'] = pd.to_datetime(
    fear_greed_index['timestamp'],
    unit='s',
    errors='coerce'
)

In [73]:
# Merge nearest by timestamp
merged_data = pd.merge_asof(
    historical_data.sort_values('Timestamp'),
    fear_greed_index.sort_values('timestamp'),
    left_on='Timestamp',
    right_on='timestamp',
    direction='nearest'
)

In [74]:
# Save merged CSV
merged_csv_path = os.path.join(csv_folder, "merged_data.csv")
merged_data.to_csv(merged_csv_path, index=False)

In [75]:
# === 4. Feature Engineering ===
merged_data["Date"] = merged_data["Timestamp"].dt.date
merged_data["abs_Size_USD"] = merged_data["Size USD"].abs()
merged_data["pnl_positive"] = (merged_data["Closed PnL"] > 0).astype(int)
merged_data["hour"] = merged_data["Timestamp"].dt.hour

In [76]:
# Daily aggregates
daily_df = merged_data.groupby("Date").agg(
    total_volume_USD=("abs_Size_USD", "sum"),
    mean_sentiment=("value", "mean"),
    avg_pnl=("Closed PnL", "mean"),
    win_rate=("pnl_positive", "mean")
).reset_index()

In [77]:
# Save daily aggregates
daily_csv_path = os.path.join(csv_folder, "daily_aggregates.csv")
daily_df.to_csv(daily_csv_path, index=False)

In [78]:
# === 5. UBM Analysis & Charts (15+) ===
fig_paths = []

def save_fig(name):
    path = os.path.join(outputs_folder, name)
    plt.tight_layout()
    plt.savefig(path)
    plt.close()
    fig_paths.append(path)

In [79]:
# 1. Univariate - PnL distribution
plt.figure(figsize=(8,5))
sns.histplot(merged_data['Closed PnL'], bins=30, kde=True)
plt.title("Distribution of Closed PnL")
save_fig("01_closed_pnl_distribution.png")

In [80]:
# 2. Univariate - Trade Size USD
plt.figure(figsize=(8,5))
sns.histplot(merged_data['abs_Size_USD'], bins=30, kde=True)
plt.title("Distribution of Trade Size (USD)")
save_fig("02_trade_size_distribution.png")

In [81]:
# 3. Univariate - Sentiment value
plt.figure(figsize=(8,5))
sns.histplot(merged_data['value'], bins=20, kde=True)
plt.title("Distribution of Fear & Greed Index Value")
save_fig("03_sentiment_value_distribution.png")

In [82]:
# 4. Bivariate - PnL by Sentiment Class
plt.figure(figsize=(10,6))
sns.boxplot(x='classification', y='Closed PnL', data=merged_data)
plt.xticks(rotation=45)
plt.title("Closed PnL by Sentiment Classification")
save_fig("04_pnl_by_sentiment_class.png")

In [83]:
# 5. Bivariate - Trade Size vs Sentiment Value
plt.figure(figsize=(8,5))
sns.scatterplot(x='value', y='abs_Size_USD', data=merged_data)
plt.title("Trade Size vs Sentiment Value")
save_fig("05_trade_size_vs_sentiment.png")

In [84]:
# 6. Bivariate - PnL vs Sentiment Value
plt.figure(figsize=(8,5))
sns.scatterplot(x='value', y='Closed PnL', data=merged_data)
plt.title("Closed PnL vs Sentiment Value")
save_fig("06_pnl_vs_sentiment.png")

In [85]:
# 7. Bivariate - Win Rate by Sentiment
winrate = merged_data.groupby("classification")["pnl_positive"].mean().sort_values()
plt.figure(figsize=(10,5))
sns.barplot(x=winrate.index, y=winrate.values)
plt.xticks(rotation=45)
plt.title("Win Rate by Sentiment Classification")
save_fig("07_winrate_by_sentiment.png")

In [86]:
# 8. Time Series - Daily Volume
plt.figure(figsize=(10,5))
sns.lineplot(x='Date', y='total_volume_USD', data=daily_df)
plt.xticks(rotation=45)
plt.title("Daily Total Volume (USD)")
save_fig("08_daily_volume.png")

In [87]:
# 9. Time Series - Mean Sentiment
plt.figure(figsize=(10,5))
sns.lineplot(x='Date', y='mean_sentiment', data=daily_df)
plt.xticks(rotation=45)
plt.title("Daily Mean Sentiment Value")
save_fig("09_daily_mean_sentiment.png")

In [88]:
# 10. Multivariate - Daily Avg PnL vs Sentiment (size=volume)
plt.figure(figsize=(8,5))
sns.scatterplot(
    x='mean_sentiment', y='avg_pnl', size='total_volume_USD',
    data=daily_df, legend=False, sizes=(20, 200)
)
plt.title("Avg PnL vs Mean Sentiment (size=Volume)")
save_fig("10_avg_pnl_vs_sentiment.png")

In [89]:
# 11. Univariate - BUY vs SELL count
plt.figure(figsize=(6,4))
sns.countplot(x='Side', data=merged_data)
plt.title("Count of BUY vs SELL Trades")
save_fig("11_buy_sell_counts.png")

In [90]:
# 12. Bivariate - Avg PnL by Hour
plt.figure(figsize=(8,5))
sns.lineplot(x='hour', y='Closed PnL', data=merged_data, estimator='mean')
plt.title("Average PnL by Hour of Day")
save_fig("12_avg_pnl_by_hour.png")

In [91]:
# 13. Correlation Heatmap
plt.figure(figsize=(10,8))
corr = merged_data.select_dtypes(include=[np.number]).corr()
sns.heatmap(corr, annot=False, cmap='coolwarm')
plt.title("Correlation Heatmap")
save_fig("13_correlation_heatmap.png")

In [92]:
# 14. Multivariate - PnL by Side and Sentiment
plt.figure(figsize=(10,6))
sns.boxplot(x='classification', y='Closed PnL', hue='Side', data=merged_data)
plt.xticks(rotation=45)
plt.title("PnL by Sentiment & Trade Side")
save_fig("14_pnl_by_side_and_sentiment.png")

In [93]:
# 15. Multivariate - Volume vs PnL vs Sentiment
plt.figure(figsize=(8,5))
sns.scatterplot(
    x='abs_Size_USD', y='Closed PnL', hue='value',
    data=merged_data, palette='coolwarm', alpha=0.7
)
plt.title("Trade Size vs PnL (color=Sentiment)")
save_fig("15_volume_vs_pnl_sentiment.png")

In [94]:
# === 6. PDF Report ===
from textwrap import wrap

pdf_path = os.path.join(root, "ds_report.pdf")
with PdfPages(pdf_path) as pdf:
    # Title page
    plt.figure(figsize=(11,8.5))
    plt.axis('off')
    plt.text(0.5, 0.7, "Web3 Trading Team - Data Science Assignment", ha='center', fontsize=20)
    plt.text(0.5, 0.6, "Candidate: LUV GOEL", ha='center', fontsize=14)
    plt.text(0.5, 0.5, f"Generated on: {datetime.now()}", ha='center', fontsize=10)
    pdf.savefig()
    plt.close()

    # Observations & Insights - Page 1
    plt.figure(figsize=(11,8.5))
    plt.axis('off')
    obs_page1 = """
Observations & Insights

1. Trade Size & PnL Patterns (Univariate)
- Closed PnL distribution is heavily skewed with most trades near zero profit/loss, a small number showing very high gains or losses.
- Trade sizes (USD) vary widely, but most are clustered in the lower range — indicating many small trades and fewer large-volume positions.
- Fear & Greed Index values are well-distributed, with clear clusters around extreme fear and extreme greed.

2. Sentiment vs Profitability (Bivariate)
- Boxplots show PnL performance differs across sentiment regimes — extreme greed days often have higher variability in profits, while extreme fear days tend to see tighter, sometimes negative distributions.
- Scatter plots suggest no strong linear correlation between sentiment index value and PnL, but clusters of positive PnL occur in both high and low sentiment ranges.
"""
    wrapped_lines = []
    for line in obs_page1.strip().split("\n"):
        wrapped_lines.extend(wrap(line, 120))
    plt.text(0.05, 0.95, "\n".join(wrapped_lines), fontsize=10, va='top')
    pdf.savefig()
    plt.close()

    # Observations & Insights - Page 2
    plt.figure(figsize=(11,8.5))
    plt.axis('off')
    obs_page2 = """
3. Win Rate Insights
- Win rate varies noticeably across sentiment categories — some sentiment states correspond to higher proportions of profitable trades.
- This indicates sentiment awareness could help in position sizing or entry timing.

4. Time-based Observations
- Daily volume fluctuates, with spikes not directly explained by sentiment shifts — possibly due to market news or events.
- Hourly patterns show certain hours have consistently higher average PnL, potentially linked to global market opening/closing times.

5. Multivariate Patterns
- Comparing PnL vs sentiment and trade side (BUY/SELL) suggests sentiment effects are slightly stronger for BUY trades.
- Correlation heatmap confirms trade size and PnL have a weak positive correlation, while sentiment’s correlation with PnL is close to zero — meaning sentiment alone isn’t predictive but may work well in combination with other factors.

---

Business Impact
1. Strategy Timing: Align larger positions with historically high win-rate sentiment states.
2. Risk Management: Reduce exposure during low win-rate sentiment regimes.
3. Trading Hours: Focus on hours with historically higher profitability.
4. Predictive Modeling: Since sentiment isn’t linearly related to PnL, use it with other features (trade size, side, hour) in more complex models for better predictions.
"""
    wrapped_lines = []
    for line in obs_page2.strip().split("\n"):
        wrapped_lines.extend(wrap(line, 120))
    plt.text(0.05, 0.95, "\n".join(wrapped_lines), fontsize=10, va='top')
    pdf.savefig()
    plt.close()

    # Add each chart
    for fp in fig_paths:
        img = plt.imread(fp)
        plt.figure(figsize=(11,8.5))
        plt.imshow(img)
        plt.axis('off')
        pdf.savefig()
        plt.close()

In [95]:
# === 7. README File ===
with open(os.path.join(root, "README.md"), "w") as f:
    f.write(
        f"Submission Structure:\n"
        f"- csv_files/: merged_data.csv, daily_aggregates.csv\n"
        f"- outputs/: {len(fig_paths)} PNG charts\n"
        f"- ds_report.pdf: PDF with Observations & Insights + charts\n"
        f"All analysis done using Python, Pandas, Matplotlib, Seaborn."
    )

print(f"Assignment package created at: {root}")
print(f"- Merged CSV: {merged_csv_path}")
print(f"- Daily Aggregates CSV: {daily_csv_path}")
print(f"- PDF Report: {pdf_path}")
print(f"- Charts saved in: {outputs_folder}")

Assignment package created at: ds_LuvGoel
- Merged CSV: ds_LuvGoel/csv_files/merged_data.csv
- Daily Aggregates CSV: ds_LuvGoel/csv_files/daily_aggregates.csv
- PDF Report: ds_LuvGoel/ds_report.pdf
- Charts saved in: ds_LuvGoel/outputs
